In [235]:
import os
import sys
import numpy as np
from random import randint

class HMM(object): 
    # base class for different HMM models
    def __init__(self, T, O, pi):
        # model is (T, O, pi) where T = Transition probs(hidden_states*hidden_states), 
        # O = Emission Probs(hidden_states*states), pi = initial distribution(hidden_states)               
        if T is None:
            print "Error: You should provide the transition matrix"
            sys.exit() # Read in parameters from the file of model_name
        if O is None:
            print "Error: You should provide the emission matrix"
            sys.exit() # Read in parameters from the file of model_name
        if pi is None:
            print "Error: You should provide the initial probability"
            sys.exit() # Read in parameters from the file of model_name           
        self.pi=pi
        self.T=T
        self.O=O
        self.M=T.shape[1]  # M:number of hidden states of the model
        self.N=O.shape[1]  # N:number of states of the model

        
    def backward(self, obs):
        # This function is for backward algorithm, suppose that T, O, pi 
        #are given, and it calculates a bwk matrix (obs*states).
        # The backward algorithm can be used to calculate the likelihood 
        #of the probability P(Y_{k+1}, ... , Y_n|t_k=C)
        #=sum_q P(Y_{k+2}, ... , Y_n|t_{k+1}=q)P(q|C)P(x_{k+1}|q)
        #The backward probability b is the probability of seeing the observations from 
        #time t + 1 to the end, given that we are in state i at time t
        self.bwk = [[0 for x in range(self.M)] for y in range(len(obs))] 
        # Initalize bwk to be empty matrix T*M
        # Initialize base cases (t == T)
        for y in range(self.M):
            self.bwk[len(obs)-1][y] = 1 
        for t in reversed(range(len(obs)-1)):
            for y in range(self.M):
                self.bwk[t][y] = sum((self.bwk[t+1][y1] * self.T[y][y1] * self.O[y1][obs[t+1]]) 
                                    for y1 in range(self.M))
                #beta_k(C)=\sum_q beta_{k+1}(q)P(q|C)P(w_{k+1}|q)
        prob = sum((self.pi[0][y]* self.O[y][obs[0]] * self.bwk[0][y]) for y in range(self.M))

        return prob 
        #This prob is the likelihood of the input obs   
 

    def forward(self, obs):
        # This function is for forward algorithm, suppose that A, B, pi are given, 
        #and it calculates a fwd matrix (obs*states).
        # The forward algorithm can be used to calculate the likelihood of the model
        #P(Y1, ... , Yn)=sum_t(\prod_i P(Y[i]|t[i])P(t[i]|t[i-1])
        self.fwd = [[0 for x in range(self.M)] for y in range(len(obs))]  
        #Initalize fwk to be empty matrix, and finally fwd is T*M
        # Initialize base cases (t == 0)
        for y in range(self.M):
            self.fwd[0][y] = self.pi[0][y] * self.O[y][obs[0]] 
            #alpha_1(q)=p(w1,t1=q)=P(t1=q|t0)*p(w1|t1=q)
        # Run Forward algorithm for t > 0
        for t in range(1, len(obs)):
            for y in range(self.M):
                self.fwd[t][y] = sum((self.fwd[t-1][y0] * self.T[y0][y] * self.O[y][obs[t]]) 
                                     for y0 in range(self.M))
                #alpha_k(q)=\sum_q1 alpha_{k-1}(q1)P(t_k=q|t_{k-1}=q1)P(w_k|t+k=q)
        prob = sum((self.fwd[len(obs) - 1][s]) for s in range(self.M))
        # The likelihood of input equals to the summation of fwd[N][t]
        return prob

    
    def viterbi(self, obs):
    #the task of determining which sequence of variables is the underlying source 
    #of some sequence of observations is called the decoding task
    #Decoding: Given as input an HMM = (A, B, pi) and a sequence of observations 
    #O = Y_1, ... Y_N, find the most probable sequence of states Q = X_1, ... X_T.
    # Goal: find the best path!
    # argmax_t P(Y1, ... Y_N, X_1, ..., X_T|A, B, pi)
        vit = [[0 for x in range(self.M)] for y in range(len(obs))] 
        # matrix
        path = {} 
        # path
        # Initialize base cases (t == 0)
        for y in range(self.M):
            vit[0][y] = self.pi[0][y] * self.O[y][obs[0]]
            path[y] = [y]
        
        # Run Viterbi for t > 0
        for t in range(1, len(obs)):
            newpath = {}
            for y in range(self.M):
                (prob, state) = max((vit[t-1][y0] * self.T[y0][y] * self.O[y][obs[t]], y0) 
                                    for y0 in range(self.M))
                vit[t][y] = prob
                newpath[y] = path[state] + [y]
            # Don't need to remember the old paths
            path = newpath
        n = 0           
        # if only one element is observed max is sought in the initialization values
        if len(obs)!=1:
            n = t
        (prob, state) = max((vit[n][y], y) for y in range(self.M))
        return (prob, path[state])

    
    
    def forward_backward(self, obs): 
        #Output matrix gamma: gamma[t][y]=P(q_t=j|Y_1, ..., Y_N,A,B,pi)
        #and tensor zi: zi[t][i][j]=P(q_t=i,q_{t+1}=j|Y_1, ..., Y_N,A,B,pi)
        gamma = [[0 for x in range(self.M)] for y in range(len(obs))]
        # this is needed to keep track of finding a state i at a time t for all i and all t
        zi = [[[0 for x in range(self.M)] for y in range(self.M)] for z in range(len(obs))]  
        # this is needed to keep track of finding a state i at a time t and j at a time (t+1) 
        #for all i and all j and all t
        # get alpha and beta tables computes
        p_obs = self.forward(obs)
        self.backward(obs)
        # compute gamma values
        for t in range(len(obs)):
            for y in range(self.M):
                gamma[t][y] = (self.fwd[t][y] * self.bwk[t][y]) / p_obs
                if t == 0:
                    self.pi[0][y] = gamma[t][y]
                #gamma[t][y]=P(q_t=j|Y_1, ..., Y_N,A,B,pi)
                #=P(q_t=j,Y_1, ..., Y_N|A,B,pi)/P(Y_1, ..., Y_N|A,B,pi)
                #=alpha_t(j)beta_t(j)/P(Y_1, ..., Y_N|A,B,pi)
                #compute zi values up to T - 1
                if t == len(obs) - 1:
                    continue
                for y1 in range(self.M):
                    zi[t][y][y1] = self.fwd[t][y] * self.T[y][y1] * self.O[y1][obs[t + 1]] * self.bwk[t + 1][y1] / p_obs
        #zi[t][i][j]=P(q_t=i,q_{t+1}=j|Y_1, ..., Y_N,A,B,pi)
        #=P(q_t=i,q_{t+1}=j,Y_1, ..., Y_N|A,B,pi)/P(Y_1, ..., Y_N|A,B,pi)
        #=alpha_t(i)a_{ij}b_j(O_{t+1})beta_{t+1}(j)/apha_t(X_T)
        return (gamma,zi)
    
    
    def baum_welch(self,obs):
        # returns model given the initial model and observations  
        #The Baum-Welch algorithm iteratively estimate the counts.
        #We will start with an estimate for the transition and observation probabilities and 
        #then use these estimated probabilities to derive better and better probabilities. 
        #We get our estimated probabilities by computing the forward probability for 
        #an observation and then dividing that probability mass among all the different 
        #paths that contributed to this forward probability.
        gamma = [[0 for x in range(self.M)] for y in range(len(obs))]
        zi = [[[0 for x in range(self.M)] for y in range(self.M)] for z in range(len(obs))]  

        # now that we have gamma and zi let us re-estimate
        (gamma,zi)=self.forward_backward(obs)
        for y in range(self.M):
            for y1 in range(self.M):
                # we will now compute new a_ij
                #a_{ij)=expected number of transitions from state i to state j/expected number 
                #of transitions from state i
                val = sum([zi[t][y][y1] for t in range(len(obs) - 1)]) #
                val /= sum([gamma[t][y] for t in range(len(obs) - 1)])
                self.T[y][y1] = val
        # re estimate gamma
        for y in range(self.M):
            for k in range(self.N): 
                # for all symbols vk
                val = 0.0
                for t in range(len(obs)):
                    if obs[t] == k :
                        val += gamma[t][y]
                val /= sum([gamma[t][y] for t in range(len(obs))])
                self.O[y][k] = val
                    #b_j(v_k)=expected number of times in state j and observing symbol vk/expected 
                    #number of times in state j
        return

In [236]:
M=randint(0,10)
N=randint(0,10)   
T_raw = np.random.random((M, M)) 
row_sums_T= T_raw.sum(axis=1)
T = T_raw / row_sums_T[:, np.newaxis]
# Get transition probability
O_raw = np.random.random((M, N))
row_sums_O = O_raw.sum(axis=1)
O = O_raw / row_sums_O[:, np.newaxis]
# Get emission probability
pi_raw = np.random.random((1, M)) 
row_sums_pi = pi_raw.sum(axis=1)
pi = pi_raw / row_sums_pi[:, np.newaxis]
# Get initial probability
hmm=HMM(T,O,pi)

In [238]:
#M=randint(0,10)
#N=randint(0,10)
#hmm=HMM(M,N)
T=randint(0,10)
observations = []
for i in xrange(0,T):
    observations.append(randint(0,N-1))
#observations=[1,0,1,1]
print "M=", M, "N=", N, "Observations = ", observations

M= 4 N= 6 Observations =  [0, 3, 0, 3, 3]


In [239]:
p1=hmm.backward(observations)
print " Fwd Prob = ", p1

 Fwd Prob =  1.86500487578e-05


In [240]:
p2=hmm.forward(observations)
print " Fwd Prob = ", p2

 Fwd Prob =  1.86500487578e-05


In [241]:
prob, hidden_states = hmm.viterbi(observations)
print "Max Probability = ", prob, " Hidden State Sequence = ", hidden_states

Max Probability =  8.17710112583e-07  Hidden State Sequence =  [1, 1, 3, 1, 1]


In [242]:
(gamma,zi)=hmm.forward_backward(observations)
print "zi= ", zi
print "gamma=", gamma

zi=  [[[0.044838056867513951, 0.12883847772164803, 0.060823296206747909, 0.011634157372551288], [0.029216654017497606, 0.21166974870675034, 0.035820915889268209, 0.089876163092098602], [0.021445583296885087, 0.027772648827908152, 0.010726608318446906, 0.019894096675005954], [0.062249072962921231, 0.19148879278696393, 0.044618456459364818, 0.0090872707984281237]], [[0.072174774960443278, 0.042291660932070672, 0.012396794218248033, 0.030886137034055856], [0.072639800037657354, 0.10731821940637207, 0.011276689778834675, 0.36853495882040627], [0.053192055098292682, 0.014047424652775403, 0.0033687758442698481, 0.081381021278489923], [0.066617186628291539, 0.041789508431245917, 0.0060460081264494406, 0.016038984752097057]], [[0.067766358665628385, 0.12830717610185419, 0.050633500614105195, 0.017916781343097064], [0.021437189365271909, 0.10233734748784644, 0.014476870438829698, 0.067195406130515972], [0.010763641872560658, 0.009184947420339722, 0.0029654078591246158, 0.010174270815777006], [0

In [243]:
hmm.baum_welch(observations)
print "The new model parameters after 1 iteration are: "
print "T = ", hmm.T
print "O = ", hmm.O
print "pi = ", hmm.pi

The new model parameters after 1 iteration are: 
T =  [[ 0.25161359  0.47203239  0.20020012  0.0761539 ]
 [ 0.1080173   0.4121652   0.05928515  0.42053234]
 [ 0.33175851  0.19885502  0.0628415   0.40654497]
 [ 0.26864787  0.5699989   0.11837006  0.04298316]]
O =  [[ 0.5404046   0.          0.          0.4595954   0.          0.        ]
 [ 0.20002386  0.          0.          0.79997614  0.          0.        ]
 [ 0.09358128  0.          0.          0.90641872  0.          0.        ]
 [ 0.69345462  0.          0.          0.30654538  0.          0.        ]]
pi =  [[ 0.42682853  0.20521318  0.01106486  0.35689343]]
